In [ ]:
pip install ogb

In [ ]:
pip install torch_geometric

In [ ]:
import os.path as osp
import time
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_undirected
from torch_geometric.nn import LabelPropagation
from torch_geometric.nn.conv.gcn_conv import gcn_norm

# -------------------------------
# Config
# -------------------------------
root = './data/Planetoid'
datasets = ['Cora', 'CiteSeer', 'PubMed']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LP_LAYERS = 1
LP_ALPHA  = 1.0

# -------------------------------
# Normalization helper
# -------------------------------
@torch.no_grad()
def normalize_adjs(edge_index, num_nodes, edge_weight=None, eps=1e-12):
    """
    Return normalized (edge_index, edge_weight) for:
    - DAD (symmetric): D^{-1/2} A D^{-1/2}
    - GCN-DAD: symmetric with self-loops (gcn_norm)
    - DA  (row):       D^{-1} A
    - AD  (col):       A D^{-1}
    Works for directed & weighted graphs.
    """
    if edge_weight is None:
        edge_weight = torch.ones(edge_index.size(1), device=edge_index.device)

    row, col = edge_index
    deg = torch.zeros(num_nodes, device=edge_weight.device).scatter_add_(0, row, edge_weight)
    deg_inv = (1.0 / deg.clamp_min(eps))
    deg_inv_sqrt = deg_inv.sqrt()

    # DAD (no self-loops)
    dad_w = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    # GCN-style DAD (with self-loops)
    gcn_ei, gcn_w = gcn_norm(
        edge_index=edge_index, edge_weight=edge_weight,
        num_nodes=num_nodes, add_self_loops=True
    )

    # DA (row-normalized)
    da_w = deg_inv[row] * edge_weight
    # AD (col-normalized)
    ad_w = deg_inv[col] * edge_weight

    return (edge_index, dad_w), (gcn_ei, gcn_w), (edge_index, da_w), (edge_index, ad_w)

# -------------------------------
# Sparse helpers
# -------------------------------
def build_sparse_from_edge_index(edge_index, num_nodes, weight=None, dtype=torch.float32):
    if weight is None:
        weight = torch.ones(edge_index.size(1), device=edge_index.device, dtype=dtype)
    return torch.sparse_coo_tensor(edge_index, weight, (num_nodes, num_nodes)).coalesce()

# -------------------------------
# LP runner
# -------------------------------
@torch.no_grad()
def run_lp(edge_index, edge_weight, y, train_mask, val_mask, test_mask, name=""):
    model = LabelPropagation(num_layers=LP_LAYERS, alpha=LP_ALPHA)
    out = model(y, edge_index, mask=train_mask, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1)

    def acc(mask):
        if mask.sum() == 0:
            return float('nan')
        return (y_pred[mask] == y[mask]).float().mean().item()

    tr, va, te = acc(train_mask), acc(val_mask), acc(test_mask)
    print(f"[{name}] Train: {tr:.4f} | Val: {va:.4f} | Test: {te:.4f}")
    return {"train": tr, "valid": va, "test": te}

# -------------------------------
# Evaluate one graph with 4 norms
# -------------------------------
def evaluate_graph(tag, edge_index, edge_weight, y, train_mask, val_mask, test_mask, num_nodes):
    results = {}
    (DAD, GCN_DAD, DA, AD) = normalize_adjs(edge_index, num_nodes, edge_weight)

    results["DAD"]     = run_lp(*DAD,     y, train_mask, val_mask, test_mask, f"{tag} LP (DAD)")
    results["GCN-DAD"] = run_lp(*GCN_DAD, y, train_mask, val_mask, test_mask, f"{tag} LP (GCN-DAD)")
    results["DA"]      = run_lp(*DA,      y, train_mask, val_mask, test_mask, f"{tag} LP (DA)")
    results["AD"]      = run_lp(*AD,      y, train_mask, val_mask, test_mask, f"{tag} LP (AD)")
    return results

# -------------------------------
# Main loop over datasets
# -------------------------------
all_results = {}

for name in datasets:
    print(f"\n================= {name} =================")
    ds = Planetoid(root=osp.join(root, name), name=name)
    data = ds[0].to(device)

    # Ensure original is undirected
    edge_index_A = to_undirected(data.edge_index, num_nodes=data.num_nodes)
    y = data.y

    print(f"nodes: {data.num_nodes}, edges: {edge_index_A.size(1)}")
    print(f"masks: train={int(data.train_mask.sum())}, val={int(data.val_mask.sum())}, test={int(data.test_mask.sum())}\n")

    # ---------- Original A ----------
    print("=== Original Graph (A) ===")
    results_A = evaluate_graph(
        f"{name}",
        edge_index_A, None,
        y, data.train_mask, data.val_mask, data.test_mask,
        data.num_nodes
    )

    # ---------- Transformed DMI = 2D + D^2 ----------
    print("\n=== Transformed Graph (SNN(beta,(1,1))) ===")
    D = build_sparse_from_edge_index(data.edge_index, data.num_nodes, dtype=torch.float32)
    torch.cuda.empty_cache()
    t0 = time.time()
    D2 = torch.sparse.mm(D, D)
    D3 = torch.sparse.mm(D, D2)
    DMI = 2*D+D2
    t1 = time.time()
    print(f"SNN(beta,(1,1)) built in {t1 - t0:.4f}s")

    # Keep DMI as-is (weighted, directed, no symmetrization)
    w = DMI.coalesce()
    ei_dmi, ew_dmi = w.indices(), w.values()

    results_DMI = evaluate_graph(
        f"{name}-DMI",
        ei_dmi, ew_dmi,
        y, data.train_mask, data.val_mask, data.test_mask,
        data.num_nodes
    )

    # ---------- Summary for this dataset ----------
    all_results[name] = {"A": results_A, "DMI": results_DMI}

# -------------------------------
# Final summary
# -------------------------------
print("\n================= SUMMARY =================")
for name in datasets:
    print(f"\n{name} (Original A):")
    for norm, accs in all_results[name]["A"].items():
        print(f"{norm:8s} -> Train: {accs['train']:.4f} | Val: {accs['valid']:.4f} | Test: {accs['test']:.4f}")
    print(f"\n{name} (SNN(beta,(1,1)):")
    for norm, accs in all_results[name]["DMI"].items():
        print(f"{norm:8s} -> Train: {accs['train']:.4f} | Val: {accs['valid']:.4f} | Test: {accs['test']:.4f}")


In [ ]:
import os.path as osp
import time
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_undirected
from torch_geometric.nn import LabelPropagation
from torch_geometric.nn.conv.gcn_conv import gcn_norm

# -------------------------------
# Config
# -------------------------------
root = './data/Planetoid'
datasets = ['Cora', 'CiteSeer', 'PubMed']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LP_LAYERS = 1
LP_ALPHA  = 1.0

# -------------------------------
# Normalization helper
# -------------------------------
@torch.no_grad()
def normalize_adjs(edge_index, num_nodes, edge_weight=None, eps=1e-12):
    """
    Return normalized (edge_index, edge_weight) for:
    - DAD (symmetric): D^{-1/2} A D^{-1/2}
    - GCN-DAD: symmetric with self-loops (gcn_norm)
    - DA  (row):       D^{-1} A
    - AD  (col):       A D^{-1}
    Works for directed & weighted graphs.
    """
    if edge_weight is None:
        edge_weight = torch.ones(edge_index.size(1), device=edge_index.device)

    row, col = edge_index
    deg = torch.zeros(num_nodes, device=edge_weight.device).scatter_add_(0, row, edge_weight)
    deg_inv = (1.0 / deg.clamp_min(eps))
    deg_inv_sqrt = deg_inv.sqrt()

    # DAD (no self-loops)
    dad_w = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    # GCN-style DAD (with self-loops)
    gcn_ei, gcn_w = gcn_norm(
        edge_index=edge_index, edge_weight=edge_weight,
        num_nodes=num_nodes, add_self_loops=True
    )

    # DA (row-normalized)
    da_w = deg_inv[row] * edge_weight
    # AD (col-normalized)
    ad_w = deg_inv[col] * edge_weight

    return (edge_index, dad_w), (gcn_ei, gcn_w), (edge_index, da_w), (edge_index, ad_w)

# -------------------------------
# Sparse helpers
# -------------------------------
def build_sparse_from_edge_index(edge_index, num_nodes, weight=None, dtype=torch.float32):
    if weight is None:
        weight = torch.ones(edge_index.size(1), device=edge_index.device, dtype=dtype)
    return torch.sparse_coo_tensor(edge_index, weight, (num_nodes, num_nodes)).coalesce()

# -------------------------------
# LP runner
# -------------------------------
@torch.no_grad()
def run_lp(edge_index, edge_weight, y, train_mask, val_mask, test_mask, name=""):
    model = LabelPropagation(num_layers=LP_LAYERS, alpha=LP_ALPHA)
    out = model(y, edge_index, mask=train_mask, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1)

    def acc(mask):
        if mask.sum() == 0:
            return float('nan')
        return (y_pred[mask] == y[mask]).float().mean().item()

    tr, va, te = acc(train_mask), acc(val_mask), acc(test_mask)
    print(f"[{name}] Train: {tr:.4f} | Val: {va:.4f} | Test: {te:.4f}")
    return {"train": tr, "valid": va, "test": te}

# -------------------------------
# Evaluate one graph with 4 norms
# -------------------------------
def evaluate_graph(tag, edge_index, edge_weight, y, train_mask, val_mask, test_mask, num_nodes):
    results = {}
    (DAD, GCN_DAD, DA, AD) = normalize_adjs(edge_index, num_nodes, edge_weight)

    results["DAD"]     = run_lp(*DAD,     y, train_mask, val_mask, test_mask, f"{tag} LP (DAD)")
    results["GCN-DAD"] = run_lp(*GCN_DAD, y, train_mask, val_mask, test_mask, f"{tag} LP (GCN-DAD)")
    results["DA"]      = run_lp(*DA,      y, train_mask, val_mask, test_mask, f"{tag} LP (DA)")
    results["AD"]      = run_lp(*AD,      y, train_mask, val_mask, test_mask, f"{tag} LP (AD)")
    return results

# -------------------------------
# Main loop over datasets
# -------------------------------
all_results = {}

for name in datasets:
    print(f"\n================= {name} =================")
    ds = Planetoid(root=osp.join(root, name), name=name)
    data = ds[0].to(device)

    # Ensure original is undirected
    edge_index_A = to_undirected(data.edge_index, num_nodes=data.num_nodes)
    y = data.y

    print(f"nodes: {data.num_nodes}, edges: {edge_index_A.size(1)}")
    print(f"masks: train={int(data.train_mask.sum())}, val={int(data.val_mask.sum())}, test={int(data.test_mask.sum())}\n")

    # ---------- Original A ----------
    print("=== Original Graph (A) ===")
    results_A = evaluate_graph(
        f"{name}",
        edge_index_A, None,
        y, data.train_mask, data.val_mask, data.test_mask,
        data.num_nodes
    )

    # ---------- Transformed DMI = 2D + D^2 ----------
    print("\n=== Transformed Graph (SNN(beta,(1,1,1))) ===")
    D = build_sparse_from_edge_index(data.edge_index, data.num_nodes, dtype=torch.float32)
    torch.cuda.empty_cache()
    t0 = time.time()
    D2 = torch.sparse.mm(D, D)
    D3 = torch.sparse.mm(D, D2)
    DMI = 3*D + 3*D2 + D3
    t1 = time.time()
    print(f"SNN(beta,(1,1,1)) built in {t1 - t0:.4f}s")

    # Keep DMI as-is (weighted, directed, no symmetrization)
    w = DMI.coalesce()
    ei_dmi, ew_dmi = w.indices(), w.values()

    results_DMI = evaluate_graph(
        f"{name}-DMI",
        ei_dmi, ew_dmi,
        y, data.train_mask, data.val_mask, data.test_mask,
        data.num_nodes
    )

    # ---------- Summary for this dataset ----------
    all_results[name] = {"A": results_A, "DMI": results_DMI}

# -------------------------------
# Final summary
# -------------------------------
print("\n================= SUMMARY =================")
for name in datasets:
    print(f"\n{name} (Original A):")
    for norm, accs in all_results[name]["A"].items():
        print(f"{norm:8s} -> Train: {accs['train']:.4f} | Val: {accs['valid']:.4f} | Test: {accs['test']:.4f}")
    print(f"\n{name} (SNN(beta,(1,1,1)):")
    for norm, accs in all_results[name]["DMI"].items():
        print(f"{norm:8s} -> Train: {accs['train']:.4f} | Val: {accs['valid']:.4f} | Test: {accs['test']:.4f}")
